In [13]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import cv2
import matplotlib.pyplot as plt
import lime
from lime import lime_image
import shap
from tf_explain.core.grad_cam import GradCAM
from tf_explain.core.integrated_gradients import IntegratedGradients
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [14]:
# Set paths
base_dir = 'Dataset_preprocessed'
train_dir = os.path.join(base_dir, 'Training_set')
test_dir = os.path.join(base_dir, 'Test_set')
classes = ['Abnormal', 'Normal']

# Image parameters
IMG_SIZE = 224
BATCH_SIZE = 32

In [15]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    # preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    # preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb'  # ResNet50 expects 3 channels
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)


Found 1640 images belonging to 2 classes.
Found 322 images belonging to 2 classes.


In [16]:
# Build pretrained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Train model
model.fit(train_generator, epochs=5, validation_data=test_generator)#, callbacks=[early_stop, checkpoint])

Epoch 1/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 258s 4s/step - accuracy: 0.5116 - loss: 0.7753 - val_accuracy: 0.6429 - val_loss: 0.6516
Epoch 2/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 210s 4s/step - accuracy: 0.5257 - loss: 0.7110 - val_accuracy: 0.6429 - val_loss: 0.6846
Epoch 3/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 186s 4s/step - accuracy: 0.4991 - loss: 0.6921 - val_accuracy: 0.4627 - val_loss: 0.6933
Epoch 4/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.5086 - loss: 0.6937 - val_accuracy: 0.3571 - val_loss: 0.6935
Epoch 5/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 308s 6s/step - accuracy: 0.4902 - loss: 0.6932 - val_accuracy: 0.3571 - val_loss: 0.6935


In [17]:
# Unfreeze some layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:143]:  # Freeze first 143 layers
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=5, validation_data=test_generator)#, callbacks=[early_stop, checkpoint])

# Save model after fine-tuning
model.save('pretrained_model/resnet50_finetuned_preprocessed.keras')
print("Model saved as 'resnet50_finetuned.keras'")

# Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/5


52/52 ━━━━━━━━━━━━━━━━━━━━ 408s 7s/step - accuracy: 0.5508 - loss: 0.7043 - val_accuracy: 0.6149 - val_loss: 0.6913
Epoch 2/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6567 - loss: 0.6319

KeyboardInterrupt: 

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
    # callbacks=[early_stop, checkpoint]
)

# Plot accuracy
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()



Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.7766 - loss: 0.4656 - val_accuracy: 0.4627 - val_loss: 1.1071
Epoch 2/10
36/52 ━━━━━━━━━━━━━━━━━━━━ 1:37 6s/step - accuracy: 0.8195 - loss: 0.4391

KeyboardInterrupt: 